In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import pyplot
from pandas.plotting import lag_plot
import plotly.offline as py
import plotly.graph_objs as go

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose 
      
from statsmodels.tools.eval_measures import rmse
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data=pd.read_csv("../input/perrin-freres-monthly-champagne-sales/Perrin Freres monthly champagne sales millions.csv")

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.describe(include='all').T

In [ ]:
#sütun adı değiştirme 
data=data.rename(columns={'Perrin Freres monthly champagne sales millions ?64-?72': 'Sales'})
data.head()

# Kayıp Değişken Analizi

In [ ]:
data.isnull().sum()

In [ ]:
data[data.isna().any(axis=1)]

In [ ]:
#etkisiz iki değişken kayıp,veriden çıkaralım.
data=data.dropna()

In [ ]:
data.isnull().sum()

In [ ]:
len(data)

# İstatiksel Analiz ve Görselleştirme

In [ ]:
data.describe()

In [ ]:
data.Month = pd.to_datetime(data.Month)
data = data.set_index("Month")
data.head()

In [ ]:
#go.scatter ile detaylı görsellştime sağlayabiliriz.Grafiğin üzerine geldiğimizde ay ve yıl bazında satışları görebiliriz.
vis=go.Scatter(x=data.index, y=data.Sales)
py.iplot([vis])

01-1964'ten 10-1972'ye kadar elimizde veri var.

**Garifk Yorum** :Seride mevsimsellik var ve trend zamanla artıyor gibi duruyor.Mevsimsellikten kaynaklı durağan bir seri değil.Durağanlığı daha iyi anlamak için ADF testi yapılmalı.

In [ ]:
decomp = seasonal_decompose(data['Sales'], model='multiplicative')
fig=decomp.plot()
fig.set_size_inches(15,10)
fig.suptitle('Decomposition plot')
plt.show()

# ADF Test #

In [ ]:
#H0:Zaman serisinin bir birim kökü var, yani seri durağan değil.
#H1:Zaman serisinin bir birim kökü yok, yani seri durağandır.

def adf_test(timeseries):
    
    print ('Dickey-Fuller Test Sonucu:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test İstatistiği','p-value','#Lags ','Gözlem sayısı'])
    for key,value in dftest[4].items():
       dfoutput['Kritik Değer (%s)'%key] = value
    print (dfoutput)


adf_test(data['Sales'])

* Durağanlık Testi sonucuna göre  p-değeri> 0.05 olduğundan H0 reddedilemez (yani kabul),verinin bir birim kökü vardır ve seri durağan değildir.(aynı zamanada test istatistiği de kritik değerden büyük)


# SARIMA modeli

Seride mevsimsellik var bu yüzden ARIMA yerine SARIMA (Mevsimsel ARIMA) kullanılacak.Hem veride tek bir değişken var ve SARIMA tek değişkenli zaman serisi verilerini mevsimsel bir bileşenle desteklediğinden uygun bir model olacaktır.

In [ ]:
pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima

auto_arima(data['Sales'], seasonal=True, m=12,start_p=0, 
           start_q=0, max_P=5, max_D=5,max_Q=5).summary()

# Train/Test Ayrımı
**Son 12 aylık veri test olarak kalan veri de train olarak ayrıldı.Tahmin 12 aylığı kapsayacak yani Ekim 1972'den Eylül 1973'e kadar.**

In [ ]:
train = data[:len(data)-12]
test= data[len(data)-12:]

In [ ]:
test

In [ ]:
arima_model = SARIMAX(train['Sales'], order = (0,0,1),
                      seasonal_order = (1,1,[],12))
arima_result = arima_model.fit()
arima_result.summary()

In [ ]:
sarima_pred = arima_result.predict(start = len(train), end = len(data)-1, typ="levels").rename("SARIMA Tahmini")

data['Sales'].plot(figsize = (15,6), legend=True)
sarima_pred.plot(legend = True);

In [ ]:
test['Sales'].plot(figsize = (15,5), legend=True)
sarima_pred.plot(legend = True);
rmse_sarima = rmse(test['Sales'], sarima_pred)
print('RMSE: %.3f' % rmse_sarima)

In [ ]:
len(data)

In [ ]:
yeni_sarima_tahmin = arima_result.predict(start =104 , end = 116, dynamic= True).rename("SARIMA Tahmini")  
data['Sales'].plot(figsize = (15,6), legend=True)
yeni_sarima_tahmin.plot(legend = True);

# LSTM ile Tahmin

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train=scaler.fit_transform(np.array(train).reshape(-1,1))
scaled_test = scaler.transform(np.array(test).reshape(-1,1))

In [ ]:
scaled_test.shape,scaled_train.shape

In [ ]:
n_input = 12
n_features= 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input,
                                batch_size=1)
model = Sequential()
model.add(LSTM(300, activation='relu', input_shape=(12, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
model.fit_generator(generator, epochs=100)

In [ ]:
losses_lstm = model.history.history['loss']
plt.figure(figsize=(12,4))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(range(len(losses_lstm)),losses_lstm);

**Test Seti ile Tahmin Edelim**

In [ ]:
rescaled_test =scaled_test.reshape(scaled_test.shape[0],scaled_test.shape[1] , 1)

lstm_tahmin = model.predict(rescaled_test)

In [ ]:
lstm_tahmin_retrans = scaler.inverse_transform(lstm_tahmin)

lstm_tahmin_testten=lstm_tahmin_retrans.reshape(lstm_tahmin_retrans.shape[0])
lstm_tahmin_testten_ser = pd.Series(lstm_tahmin_testten, index=test.index)

In [ ]:
lstm_tahmin_testten_ser

In [ ]:
lstm_tahmin_testten_ser.plot(figsize = (15,5), legend=True)
test['Sales'].plot(legend = True);
rmse_lstm = rmse(test['Sales'], lstm_tahmin_testten_ser)
print('RMSE: %.3f' % rmse_lstm)

**Ekstrapolasyon (Tek değişkenli zaman serisi tahmini) ile Tahmin Edelim**

In [ ]:
lstm_tahmin_scaled=list()
batch = scaled_train[-12:]
current_batch = batch.reshape((1, 12, 1))
for i in range(len(test)):   
    lstm_tahmin = model.predict(current_batch)[0]
    lstm_tahmin_scaled.append(lstm_tahmin) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_tahmin]],axis=1)

In [ ]:
lstm_tahmin_ekst = scaler.inverse_transform(lstm_tahmin_scaled)

lstm_tahmin_reshape=lstm_tahmin_ekst.reshape(lstm_tahmin_ekst.shape[0])
lstm_tahmin_ekst_ser=pd.Series(lstm_tahmin_reshape,index=test.index)

In [ ]:
lstm_tahmin_ekst_ser.plot(figsize = (15,5), legend=True)
test['Sales'].plot(legend = True);
rmse_lstm = rmse(test['Sales'], lstm_tahmin_ekst_ser)
print('RMSE: %.3f' % rmse_lstm)

# Prophet ile Tahmin

In [ ]:
data

In [ ]:
df = data.copy()
df = df.reset_index()
df.columns = ['ds','y'] 

train_pr = df.iloc[:len(data)-12]
test_pr = df.iloc[len(data)-12:]

from fbprophet import Prophet

model_p = Prophet()
model_p.fit(train_pr)
future = model_p.make_future_dataframe(periods=12,freq='MS')

prophet_pred = model_p.predict(future)
prophet_pred.tail()

In [ ]:
prophet_pred = pd.DataFrame({"Tarih" : prophet_pred[-12:]['ds'], 
                             "Prophet_Tahmin" : prophet_pred[-12:]["yhat"]})
prophet_pred = prophet_pred.set_index("Tarih")
prophet_pred.index.freq = "MS"
prophet_pred

In [ ]:
prophet_pred['Prophet_Tahmin'].plot(figsize = (15,5), legend=True)
test['Sales'].plot(legend = True);
rmse_lstm = rmse(test['Sales'], prophet_pred['Prophet_Tahmin'])
print('RMSE: %.3f' % rmse_lstm)

# Sonuçları Gösteren DataFrame Oluşturma

In [ ]:
from pandas.tseries.offsets import DateOffset

tahmin=[data.index[-1]+ DateOffset(months=x)for x in range(0,13)]
tahmin_datası=pd.DataFrame(index=tahmin[1:])

tahmin_datası

In [ ]:
tahmin_datası['SARIMA_Tahmin']=yeni_sarima_tahmin
tahmin_datası['LSTM_Tahmin']=lstm_tahmin_ekst
tahmin_datası['Prophet_Tahmin']=prophet_pred['Prophet_Tahmin'].values

In [ ]:
tahmin_datası

*Sonuç olarak en az hata değerini SARIMAX modelinin sonucu vermektedir.LSTM ise Prophet modelinden daha iyi sonuç vermiştir.*